# Assignment 04 - (*IV only*) Motion Planning

## Goals
Using the detections of the pedestrians and the measured vehicle pose, you will implement the **motion planning** using control parameterization and optimization (cf. [Practicum 3 IV](../practicum3_iv/practicum3_iv.ipynb)).

## Input

Input will be the `pedestrian_states`, `vehicle_positions` and `frame_indices`, as provided by us within `tracked_pedestrian_states.npz`.
We will use the function `from solution_helpers import load_pedestrian_states` to load the data.

## Output
- Answers to the questions within this notebook
- Plots, visualizations and videos within this notebook
- The metrics generated in the final cell

## Your Task
In Practicum 3, we used a trajectory optimization method to generate a safe trajectory for the vehicle. In this task, our goal is to generate safe trajectories using real sensor data as well as the results from previous notebooks. We provide part of the code that was available in Practicum 3. 

The layout of this assignment is as follows:
-  We first visualize vehicle and pedestrian data for all time steps
-  Then, we pick a *scenario*, i.e., a particular time step for which we will plan a motion. There are four scenarios of increasing difficulty.
-  The next cell contains basic code for parameterizing the control input with control profiles.
-  The following cell contains necessary functions for trajectory optimization, similar to Practicum 3, with minor changes. You will need to complete these functions where indicated.

At this point we evaluate the controller,

- First, on a single scenario (selected by `scenario_idx`) for which the candidates and selected trajectory are visualized and animated. You need to define the goals of the candidates here as well as their layout in the planning space.
- Then we evaluate your solution on a set of metrics for all scenarios. These metrics are: traveled distance, collisions and near collisions, offset from the road center, and time spend offroad.

The number of credits obtained in this assignment depends on how well your solution tackles the provided scenarios based on the defined metrics. Your solution from Practicum 3 may be sufficient for the simpler two scenarios, but will likely fail on the latter two scenarios. In addition, you may have to change your cost function in comparison to Practicum 3. A pass grade can be obtained by 
* a score of at least 95 on scenario 0, AND 
* a score of at least 1 on the scenario 1 (implying no collisions and limited offroad time)

For a good grade, your solutions needs to prevent collision and offroad time to score at least 1 on scenario 2 and 3. As we want our motion planner to be applicable to different scenarios in general, **it is not allowed to tune the controller differently per scenario. In addition, the solution should remain efficient, i.e., some credits may be deducted if the number of generated candidates is excessive.**

To accomodate for better solutions to the motion planning problem, two parts of the code have changed compared to practicum 3: 
- The vehicle in this exercise is controlled by velocity (`u_vel`) rather than acceleration.
- The `SteeringProfile` class is rewritten to extend from the base class `ControlProfile`.

**Hint:** To pass the last two scenarios, steering maneuvres may not be enough (i.e., the candidates in Practicum 3). You will have to combine different types of profiles to get a wider range of possible vehicle plans and choose the best from these.

### Q 04.1 Specification of intended solution
Before diving into the code, please describe how you will implement a safe and efficient motion planner accounting for the pedestrians. Here are some questions to stimulate the specification of your solution:
1. How will you construct each candidate trajectory?
2. How will you divide candidates over the planning space?
3. How will you trade-off safety and efficiency when picking candidates?
4. If you plan to pass the latter two scenarios, what will you change throughout the planner to avoid collisions while staying on the road?

More sophisticated/creative/robust/efficient solutions will earn you more credits.

### A 04.1
**Your Answer: (maximum 400 words)**

1.

2.

3.

4.

YOUR ANSWER HERE

In [ ]:
# some magic to ease implementing within jupyter notebooks
from IPython import get_ipython

ipython = get_ipython()
if ipython:
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

# import functions of practicum3_iv.ipynb notebook
import ipynb.fs.defs.practicum3_iv as practicum3_iv

Let's load the `pedestrian_states`, `vehicle_states` and `frame_indices` from disk:

In [ ]:
from solution_helpers import load_pedestrian_states

pedestrian_states, vehicle_states, frame_indices = load_pedestrian_states()

# Load pedestrian data from the previous steps
n_timesteps, n_tracks, state_dim = pedestrian_states.shape
print("Data has {} timesteps and {} tracks".format(n_timesteps, n_tracks))

In [ ]:
assert state_dim == len(["x", "y", "vx", "vy"])  # to show what state resembles
assert n_timesteps == 116, n_timesteps

Let's visualize the loaded pedestrian tracks and the vehicle position over time.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from interfaces import vehicle
from ipywidgets import IntSlider, fixed, interact
from planning_visualization import plot_ped_positions_per_timestep

dt = 0.1
plot_ped_f = lambda timestep: plot_ped_positions_per_timestep(
    timestep, pedestrian_states, vehicle_states, vehicle, n_tracks, dt
)

obj = interact(plot_ped_f, timestep=IntSlider(min=1, max=n_timesteps - 1, step=1, value=0))

## Defining the scenario

Now, let us assume that the vehicle is at a *particular* time step `timestep_to_solve`. We have an estimate of the position and velocity of the pedestrians at this time, as obtained in previous notebooks. Our goal is to plan a safe motion around the pedestrians in the near future. To do so, we will assume that pedestrians move at a constant velocity, i.e., we compute their positions as

\begin{align}
x_{k+1} &= x_k + v_0 \cos{\theta_0}\\
y_{k+1} &= y_k + v_0 \cos{\theta_0}
\end{align}

We model their collision region as a disc with a radius of $1$ m (see `pedestrian_radius`). We have preprogrammed some interesting scenarios, which you can select with `scenario_idx`. The predicted movement of pedestrians is visualized below.
<a id="scenario_selection"></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from interfaces import dynamic_obstacle, road, vehicle
from ipywidgets import IntSlider, fixed, interact
from planning_visualization import plot_scene

# The pedestrian radius
pedestrian_radius = 1.0

# Vehicle dimensions, in case you need them
vehicle_length = 4.5
vehicle_width = 2.0

# Pick a scenario to test (0-3)
scenario_idx = 0

def initialize_scenario(scenario_idx, supress_output=False):
    assert scenario_idx <= 3

    if scenario_idx == 0: 
        t = 2 # Scenario (Easy) A pedestrian is near the road
    elif scenario_idx == 1:
        t = 56 # Scenario (Medium) A pedestrian is walking on the road
    elif scenario_idx == 2:
        t = 65  # Scenario (Hard) A pedestrian is crossing in front of the vehicle
    elif scenario_idx == 3: 
        t = 63 # Scenario (Hard) A pedestrian is walking in front of the vehicle
        
    timestep_to_solve = t

    if not supress_output:
        print("Loading scene data...")
        print("\tSelected timestep: {}".format(timestep_to_solve))

    # Load pedestrians
    pedestrians = []
    for track_idx in range(n_tracks):
        if np.all(np.isfinite(pedestrian_states[timestep_to_solve, track_idx, 0:2])):
            pedestrians.append(dynamic_obstacle(pedestrian_states[timestep_to_solve, track_idx, :], track_idx))
            # print(pedestrians[-1])

    if not supress_output:
        print("\tTracking {} pedestrians".format(len(pedestrians)))

    # Load the vehicle data
    scene_vehicle = vehicle(vehicle_states[timestep_to_solve, :], vehicle_states[timestep_to_solve - 1, :], dt)
    if not supress_output:
        print("\tVehicle at (x = {}, y = {})".format(scene_vehicle.state.x, scene_vehicle.state.y))
        print("\tVehicle initial velocity is {} m/s".format(scene_vehicle.state.v))

        print("\tRoad defined.")
    road_length = scene_vehicle.state.v * dt * 100  # Length if we drove consistently (road limit)
    scene_road = road(
        scene_vehicle.state, radius=1e6, length=road_length, width=4, steps=100
    )  # Initialize the straight road

    return timestep_to_solve, pedestrians, scene_vehicle, scene_road


# Initialize
timestep_to_solve, pedestrians, scene_vehicle, scene_road = initialize_scenario(scenario_idx)

print("Plotting the scene...")
fig, ax = plt.subplots(figsize=(12, 10))
plot_scene(ax, scene_vehicle, pedestrians, scene_road)
ax.legend(loc="upper left")

print("Done.")

# Your Motion Planner

## Control Parameterization

**Please carefully check the available functionality in the `ControlProfile` base class.**

Then, define your control parameterization in the following cell to visualize.





In [ ]:
import copy
import math

import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import scipy.interpolate
from interfaces import dynamic_obstacle_state, make_road_xy, road, vehicle_state


# This is a base class for control parameterization
# You may want to use this class for an additional parameterization
# In that case, note that a visualization is available which can be used to check a control parameterization:
    # my_test_profile = MyProfile(test_param)
    # my_test_profile.visualize()
    # plt.show()
class ControlProfile:
    def __init__(self):
        self.params = None
        self.profile = None

    def construct(self):
        """
        Construct a control profile, i.e., a map from 0 <= s <= 1 to an input u
        Output:
            self.profile : a mapping function (i.e. self.profile = lambda s : <your function of s>)
        """
        pass

    def evaluate(self, sfrac):
        """
        Evaluates the control profile at a relative distance along the trajectory
        Input:
            sfrac : A float or np.array of values of 0 <= s <= 1
        Output:
            The value resulting from the profile at s
        """
        return self.profile(sfrac)

    def visualize(self):
        """
        Call this function to visualize the profile in a plot
        """
        if self.profile is not None:
            s_range = np.linspace(0.0, 1.0, 25)

            fig, ax = plt.subplots(figsize=(10, 5))

            ax.plot(s_range, self.evaluate(s_range), "k-", label="spline")
            ax.grid()
            ax.set_xlim(0.0, 1.0)

            ax.set_xlabel("fraction of travelled distance")
            ax.set_ylabel(type(self).__name__)


class SteeringProfile(ControlProfile):

    # The initialization of the steering profile
    def __init__(self, k0, k1, k2):

        # Save parameters
        self.k0 = k0
        self.k1 = k1
        self.k2 = k2

        self.construct()

    # Tip: use scipy's 'scipy.interpolate.CubicSpline' function
    # See https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.html
    def construct(self):
        # Todo: implement your steering function here
        # self.profile = 
        # YOUR CODE HERE
        raise NotImplementedError()

    # Custom visuals for the steering profile (like in Practicum 3)
    def visualize(self):
        long_fracs = np.linspace(0, 1, 25)

        fig, ax1, ax2 = practicum3_iv.animate_steering_init(self, long_fracs, self.evaluate(long_fracs))

        ims = []
        for j in range(long_fracs.size):
            ims.append(
                practicum3_iv.animate_steering_profile_plot(
                    ax1, ax2, steer_profile, long_fracs[j], self.evaluate(long_fracs[j])
                )
            )

        anim = animation.ArtistAnimation(fig, ims, repeat=False, interval=100)
        plt.close()
        return anim


# YOUR CODE HERE
raise NotImplementedError()


# Set the initial state and goal state for the vehicle
init_state = scene_vehicle.state
goal_state = scene_vehicle.state.get_goal_state(scene_road, 1.0, 0.0, init_state)

# Initialize candidate parameters
k1 = init_state.kappa
k2 = init_state.kappa
path_length = scene_road.rlength

if ipython:
    ipython.magic("matplotlib inline")
from IPython.display import HTML  # For animations in the notebook

steer_profile = SteeringProfile(-1.0, 0.2, -0.4)  # Example values (default is straight)
anim = steer_profile.visualize()
HTML(anim.to_html5_video())

## Optimization and Candidate Costs

The functions below are modified versions of those in IV Practicum 3, accomodating for the velocity profile and for handling multiple pedestrians. Complete the functions below. Note that
- The last argument of `euler_integrate_motion`, `other_profile`, can be used to supply additional control profiles from `curv_param_state_predict`, if you want to. If not, you can leave the argument empty.

- The last argument of `curv_param_state_predict`, `extra_params`, is available for supplying parameters that you do not want to optimize.

- The cost in `compute_trajectory_cost` needs to be adapted for the pedestrians and their predictions. You may also want to change your cost in comparison to IV Practicum 3. For example, the collision region in IV Practicum 3 was considering the vehicle to be a single disc, which is inaccurate. 

In [ ]:
from interfaces import candidate, road, vehicle_state
import scipy.optimize

# Simulate the motion of the vehicle subject to a control parameterization
def euler_integrate_motion(init_state, dt, path_length, steering_profile, other_profile=None):

    motion_model = lambda state, u_vel, u_steer, dt: state.motion_model(u_vel, u_steer, dt)

    t = 0.0
    state = copy.deepcopy(init_state)
    states = np.array([])

    # Modification: We may not reach our path length.
    # This time limit ensures that our optimization still retains useful information about the trajectory.
    T = 10.0
    nsteps = math.floor(T / dt)

    s = 0.0
    v = init_state.v
    for step in range(nsteps):
        t = t + dt
        dt = min(dt, T - t)  # last time step might use smaller dt

        # Todo: Compute the control inputs and simulate the model
        # You can evaluate the motion model by calling the `motion_model` lambda as defined above
        # This will change the state INTERNALLY, i.e., it does not have outputs
        # Note that the inputs are u_vel, u_steer

        # YOUR CODE HERE
        raise NotImplementedError()

        s += state.v * dt  # Keeps track of distance traveled

        states = np.append(states, copy.copy(state))

    # Return the array of states
    return states


# Parameterize the control input and obtain the resulting trajectory
def curv_param_state_predict(init_state, dt, params, extra_params=None):

    k0 = init_state.kappa

    states = np.array([init_state])  # Dummy

    # Todo: Define the control parameterization and compute the resulting states
    # Note that the profile class names are capitalized (e.g., SteeringProfile)

    # YOUR CODE HERE
    raise NotImplementedError()

    # Return the resulting states
    return states



def optimize_control_parameters(init_state, goal_state, dt, init_params, extra_params=None,\
                                bounds=None, surpress_output=True):

    if surpress_output:
        verbose_setting = 0
    else:
        verbose_setting = 2
        
    # We run without bounds in this practicum!
    if bounds is None:
        lb = ()
        ub = ()
        for i in range(len(init_params)):
            lb += (-np.Inf,)
            ub += (np.Inf,)
        bounds = [lb, ub]

    # Run non-linear least-squares optimization to optimize our parameters p
    params = scipy.optimize.least_squares(
        cost_function,
        init_params, 
        bounds=bounds,
        args=(init_state, goal_state, dt, extra_params),
        loss="linear",
        verbose=verbose_setting,
        max_nfev=100,
    )

    # Return the optimized paramers
    return params.x

# this is the error function
def cost_function(params, init_state, goal_state, dt, extra_params):

    # TODO: Implement your error value here
    err = 0  # Dummy
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return err

# Evaluate a candidate trajectory by comparing each timestep
def compute_trajectory_cost(pedestrians, candidate):

    # number of timesteps
    nsteps = min(candidate.states.size, len(pedestrians[0].prediction))

    costs_per_timestep = np.ones([nsteps]) * np.nan

    # compute cost for each time step
    for step in range(nsteps):

        costs_per_timestep[step] = 0  # Dummy

        # Todo: Compute here the cost per timestep
        # Predicted pedestrians can be obtained with
        # "pedestrian.prediction[step, 0]", "pedestrian.prediction[step, 1]"

        # YOUR CODE HERE
        raise NotImplementedError()

    total_cost = np.array([0])  # dummy

    # Todo: define the total cost
    # YOUR CODE HERE
    raise NotImplementedError()

    # Some functions to test the cost function
    assert costs_per_timestep.size == nsteps
    assert total_cost.size == 1

    # Return the total cost and the cost per time step
    return total_cost, costs_per_timestep

## Candidate Construction and Visual Validation

The following cell should construct your candidates in `candidates` based on your definition of the candidate goals.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from interfaces import candidate
from planning_visualization import animate_final_trajectory, plot_pedestrians


def construct_and_optimize_candidates():
    
    # Set the initial state and goal state for the vehicle
    init_state = scene_vehicle.state
    goal_state = scene_vehicle.state.get_goal_state(scene_road, 1.0, 0.0, init_state)

    # Initialize candidate parameters
    k1 = init_state.kappa
    k2 = init_state.kappa
    path_length = scene_road.rlength

    # TODO: Define the goals of your candidates and set the initial parameters
    # Hint: You can use "scene_vehicle.state.get_goal_state(road, length, lat_offset, init_state)
    # to compute the goal of a candidate.
    # Herein,
        # "length" (0. < length < 1.) is the fraction of the road traveled, and 
        # "lat_offset" (-1. < lat_offset < 1.) is the lateral deviation from the road center
            
    # lat_offsets = ?
    # init_params = ?
    # ...?

    # YOUR CODE HERE
    raise NotImplementedError()

    # Optimize the initial parameters
    init_params = optimize_control_parameters(
        init_state, goal_state, dt, init_params, curv_param_state_predict, bounds=None, surpress_output=True
    )

    candidates = np.array([])
    print("Candidate: ")
    for i, lat_offset in enumerate(lat_offsets):
        if i > 0:
            print(".")

        # Todo: Construct and optimize candidates

        # You can add candidates by computing their states, then setting
            # new_candidate = candidate(lat_offset, long_offset, params, states)
        # And appending new_candidate to "candidates"

        # YOUR CODE HERE
        raise NotImplementedError()

    print(".")

    # ---
    # We evaluate candidate tracks
    for cidx in range(candidates.size):
        c = candidates[cidx]
        [total_cost, cost_per_timestep] = compute_trajectory_cost(pedestrians, c)

        candidates[cidx].cost_per_timestep = cost_per_timestep
        candidates[cidx].total_cost = total_cost

    return candidates, init_state, goal_state


candidates, init_state, goal_state = construct_and_optimize_candidates()

if ipython:
    ipython.magic("matplotlib inline")
from IPython.display import HTML  # For animations in the notebook

anim, candidate_cost, s_candidate, states = animate_final_trajectory(scene_vehicle, pedestrians, scene_road, candidates)
plt.close()
HTML(anim.to_jshtml())

Use this link to go back to the [scenario selection](#scenario_selection).

## Numerical Validation

You can evaluate your solution on all scenarios by running the cell below.

The computed score awards points for the percentage of road traveled (i.e., between $0$ and $100$), then penalizes the following behavior:
* Any collision results in $-200$ points
* Intruding within $3.0$m of a pedestrian yields at most $50$ penalty points up until the collision point.
* Going off road yields $150/4$, $150/2$ or $150$ penalty points the longer the vehicle drives offroad
* Deviation from the road center line is penalized with at most $10$ points scaling linearly.


In [ ]:
from interfaces import find_best_candidate
from validation_metrics import check_offroad, lateral_offset, min_dist_to_obstacles, progress_to_goal, score

saved_scenario = scenario_idx

scores = []
for validation_scenario_idx in range(4):
    print("Generating Trajectories for Scenario {}...".format(validation_scenario_idx))

    timestep_to_solve, pedestrians, scene_vehicle, scene_road = initialize_scenario(validation_scenario_idx, True)

    candidates, init_state, goal_state = construct_and_optimize_candidates()
    s_candidate, states, candidate_cost, cidx = find_best_candidate(candidates)

    max_travel = scene_vehicle.state.get_goal_state(scene_road, 1.0, 0.0, init_state)
    validation = dict()

    validation["min_distance"], validation["collisions"] = min_dist_to_obstacles(pedestrians, s_candidate)
    validation["progress"] = progress_to_goal(s_candidate, max_travel)
    validation["lat_offset"] = lateral_offset(s_candidate, scene_road)
    validation["offroads"] = check_offroad(s_candidate, scene_road, dt)

    print("\n----- Validation -----")
    print("Minimum Distance to Pedestrians: {:.2f} m".format(validation["min_distance"]))
    print("Collisions:\t\t\t {:d}".format(validation["collisions"]))
    print("Progress to Goal:\t\t {:.2f} %".format(validation["progress"] * 100))
    print("Lateral Offset from Road Center: {:.2f} %".format(validation["lat_offset"] * 100))
    print("Time Offroad:\t\t\t {:.2f}s".format(validation["offroads"]))

    student_score = score(validation)
    scores.append(student_score)
    print("\nCompiled Score: {:.2f}".format(student_score))
    print("----------------------\n")

print("Scores:\n")
for idx, score in enumerate(scores):
    print("\tScenario {:d}: {:.2f}".format(idx, score))
    
scenario_idx = saved_scenario
timestep_to_solve, pedestrians, scene_vehicle, scene_road = initialize_scenario(scenario_idx, True)

In [ ]:
# DO NOT DELETE THIS CELL!

### Q 04.2 Motion Planning Performance
Please reflect on the behavior of your motion planner, in particular:

1. How would you characterize the driving style of your motion planner, i.e., how does your planner trade-off safety and performance? 
2. Do you think that your solution is robust to other scenarios?
3. What shortcomings do you think are inherent to your approach?

### A 04.2
**Your answer:** (maximum 300 words)
1. 
2. 
3. 

YOUR ANSWER HERE

### Q 04.3 Future Work
How can improve your method even more, i.e., if you had more time at your disposal?

### A 04.3
**Your answer:** (maximum 200 words)


YOUR ANSWER HERE

In [ ]:
# DO NOT DELETE THIS CELL!

In [ ]:
# DO NOT DELETE THIS CELL!

In [ ]:
# DO NOT DELETE THIS CELL!

In [ ]:
# DO NOT DELETE THIS CELL!

## Great Job!

By completing the final assignment you have implemented essential parts of the pipeline that make a self-driving car *self-driving*.